<a href="https://colab.research.google.com/github/KrisNguyen135/Project-Euler/blob/master/written_solutions/p704.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Euler 704: Factors of Two in Binomial Coefficients

Link to the problem prompt: [https://projecteuler.net/problem=704](https://projecteuler.net/problem=704)

Due to [Kummer's Theorem](http://www.aquatutoring.org/KummerTheoremLucasTheorem.pdf), $g(n, m)$ can be computed as the number of carries when $(n - m)_2$ is added to $m_2$, where $x_2$ is the binary representation of a positive integer $x$. This is exactly equal the sum of digits of $(n - m)_2$ plus the sum of digits of $m_2$ minus the sum of digits of $n_2$.

From here, it can be proven using induction that if $F(n) = \max \{ g(n, m): 0 \leq m \leq n \}$, then:

$$F(n) = k \enspace \text{if} \enspace 2 | n \enspace \text{else} \enspace F(n) = F \left( \lfloor \frac{n - 1}{2} \rfloor \right)$$

From here $S$ can be computed in a brute-force way.

In [0]:
from math import log2
from tqdm import tqdm

In [0]:
def count_n_carries(a, b):
    def get_digit_sum(str_x):
        return sum(int(d) for d in str_x)

    bin_a = bin(a)[2:]
    bin_b = bin(b)[2:]

    bin_sum = bin(a + b)[2:]

    return get_digit_sum(bin_a) + get_digit_sum(bin_b) - get_digit_sum(bin_sum)


def g(n, m):
    return count_n_carries(n - m, m)


def F(n):
    return max(g(n, m) for m in range(n + 1))


def F_v2(n, k=None):
    if k is None:
        k = int(log2(n))
    bin_n = bin(n)[2:]

    if bin_n[-1] == '0':
        return k
    
    if '0' not in bin_n:
        return 0
    
    return F_v2((n - 1) // 2)


def S(n):
    return sum(F_v2(i) for i in range(1, n + 1))

To compute $S$ efficiently, we first find the analytical expression for a special case of $S$. Specifically, define $H(k) = S(2^k - 1)$, we can derive the recursion $H(k + 1) = H(k) + k \cdot 2^k - 2^k + 1$, which, after some algebraic manipulations, yields $H(k) = (k - 3) \cdot 2^k + k + 3$. So,

$$S(N) = H(k) + \sum_{n = 2^k}^N F(n)$$

where $k = \lfloor \log_2(N) \rfloor$. Since $H$ can be computed efficiently, we only need consider the summation term. Recall that $F(n) = k$ if $n$ is even, so the summation can be computed using dynamic programming, implemented by function `S_v3`.

In [0]:
def H(k):
    return (k - 3) * 2 ** k + k + 3


def S_v2(n):
    k = int(log2(n))

    return H(k) + sum(F_v2(i, k) for i in range(2 ** k, n + 1))


def S_v3(n):
    def sub_S(sub_n, sub_k=None):
        if sub_k is None:
            sub_k = int(log2(sub_n))
        if sub_n == 2 ** sub_k:
            return sub_k

        return sub_k * ((sub_n - 2 ** sub_k) // 2 + 1) + sub_S((sub_n - 1) // 2)

    k = int(log2(n))
    
    return H(k) + sub_S(n, k)

In [19]:
S_v3(100), S_v3(10 ** 7)

(389, 203222840)

In [20]:
S_v3(10 ** 16)

501985601490518144